# 16장. 코루틴

이 장에서는 다음 내용들을 설명한다.

* 제너레이터를 코루틴으로 만드는 방법
* 코루틴으로 작동하는 제너레이터의 동작과 상태
* 제너레이터 객체의 `close()`와 `throw()` 메서드를 통해 호출자가 코루틴을 제어하는 방법
* 종료할 때 코루틴이 값을 반환하는 방법
* 새로운 `yield from` 구문의 사용법과 의미
* 사용 예: 시뮬레이션의 동시 활동을 관리하기 위한 코루틴

## 16.1 코루틴은 제너레이터에서 어떻게 진화했는가?

...

## 16.2 코루틴으로 사용되는 제너레이터의 기본 동작

다음 예제는 코루틴의 동작을 보여준다.

In [7]:
def simple_coroutine(): # 코루틴은 본체 안에 yield문을 가진 일종의 제너레이터 함수로 정의된다
    print('-> coroutine started')
    x = yield # yield문을 표현식에 사용한다.
    print('-> coroutine received:', x)

In [8]:
my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x7f09e82d9930>

In [9]:
next(my_coro) # 제너레이터를 yield문까지 실행함으로써 데이터를 전송할 수 있는 상태를 만든다.

-> coroutine started


In [10]:
my_coro.send(42) # send() 메서드를 호출해서 본체 안의 yield문의 값을 42로 만든다.
# 이제 코루틴이 실행되어서 다음 yield문이 나오거나 종료될 때까지 실행된다.

-> coroutine received: 42


StopIteration: 

제어 흐름이 코루틴 본체의 끝에 도달하므로 일반적인 제너레이터와 마찬가지로 `StopIteration` 예외를 발생시킨다.

코루틴은 다음 네 가지 상태를 가지며, `inspect.getgeneratorstate()` 함수를 이용해 현재 상태를 알 수 있다.

* `GEN_CREATED` : 실행을 위해 대기하고 있는 상태
* `GEN_RUNNING` : 현재 인터프리터가 실행하고 있는 상태
* `GEN_SUSPENDED` : 현재 yield 문에서 대기하고 있는 상태
* `GEN_CLOSED` : 실행이 완료된 상태

`send()` 메서드에 전달한 인수가 대기하고 있는 yield 표현식의 값이 되므로, 코루틴이 대기 상태에 있을 때는 `my_coro.send(42)`와 같은 형태로만 호출할 수 있다. 그러나 코루틴이 아직 기동되지 않은 `GEN_CREATED` 상태인 경우에는 `send()` 메서드를 호출할 수 없다. 그래서 코루틴을 처음 활성화하기 위해 `next(my_coro)`를 호출한다.

코루틴 객체를 생성한 직후 `None`이 아닌 값을 전달하려고 하면 다음과 같이 오류가 발생한다.

In [11]:
my_coro = simple_coroutine()
my_coro.send(1729)

TypeError: can't send non-None value to a just-started generator

처음 `next(my_coro)`를 호출할 때 코루틴을 기동한다. 즉 처음 yield문까지 실행을 진행한다. 

yield문이 두 번 이상 나오는 예제를 보자.

In [12]:
def simple_coro2(a):
    print('-> Started: a=', a)
    b = yield a
    print('-> Received: b=', b)
    c = yield a + b
    print('-> Received: c=', c)

In [21]:
my_coro2 = simple_coro2(14)

In [22]:
from inspect import getgeneratorstate
getgeneratorstate(my_coro2) # 아직 실행안됨

'GEN_CREATED'

In [23]:
next(my_coro2) # 첫번째 yield문까지 진행하면서 메세지를 출력하고 a의 값을 생성한 후 b에 값이 할당될 때까지 기다린다.

-> Started: a= 14


14

In [24]:
getgeneratorstate(my_coro2) # 코루틴이 yield문에서 대기하고 있는 상태

'GEN_SUSPENDED'

In [25]:
my_coro2.send(28) # 중단된 코루틴에 28을 보내면 yield문의 값은 28로 평가되고 이 값이 b에 할당된다. 그후 a+b의 값(42)가 생성된다.

-> Received: b= 28


42

In [26]:
my_coro2.send(99) # yield는 99로 평가되고 코루틴은 종료된다.

-> Received: c= 99


StopIteration: 

In [27]:
getgeneratorstate(my_coro2) # 종료된 상태

'GEN_CLOSED'

코루틴의 할당문에서는 실제 값을 할당하기 전에 = 오른쪽 코드를 실행한다. 즉 `b = yield a`에서는 나중에 호출자가 값을 보낸 후에야 변수 b가 설정된다. 